# Dataset Extraction

### Date: January 12, 2020

In [1]:
import warnings
warnings.filterwarnings('ignore')
import concurrent.futures
import time
from multiprocessing import Pool
import PyPDF4
import os
os.chdir(os.curdir)
os.chdir("..")
import fitz

In [2]:
path_1 = 'pdf_samples/nature1.pdf'
path_2 = 'pdf_samples/nature2.pdf'
path_3 = 'pdf_samples/nature3.pdf'
path_4 = 'pdf_samples/nature4.pdf'
path_5 = 'pdf_samples/nature5.pdf'

In [3]:
import re
import textract
#read the content of pdf as text
text = textract.process(path_1)
#use four space as paragraph delimiter to convert the text into list of paragraphs.
prg = re.split('\s{4,}', text.decode("utf-8"))

In [75]:
url = 'https://arxiv.org/pdf/1906.08237.pdf'

In [83]:
from urllib.request import urlretrieve
urlretrieve(url, "download.pdf")

('download.pdf', <http.client.HTTPMessage at 0x10c6b2d10>)

In [84]:
path = 'download.pdf'

In [86]:
doc = open(path, mode='rb')

In [87]:
pdf_document = PyPDF4.PdfFileReader(doc)
first_page = pdf_document.getPage(0)
first_page.extractText()

'XLNet:GeneralizedAutoregressivePretraining\nforLanguageUnderstanding\nZhilinYang\n\n1\n,ZihangDai\n\n12\n,YimingYang\n1\n,JaimeCarbonell\n1\n,\nRuslanSalakhutdinov\n1\n,QuocV.Le\n2\n1\nCarnegieMellonUniversity,\n2\nGoogleAIBrainTeam\n{zhiliny,dzihang,yiming,jgc,rsalakhu}@cs.cmu.edu,qvl@google.com\nAbstract\nWiththecapabilityofmodelingbidirectionalcontexts,denoisingautoencoding\nbasedpretraininglikeBERTachievesbetterperformancethanpretrainingap-\nproachesbasedonautoregressivelanguagemodeling.However,relyingoncorrupt-\ningtheinputwithmasks,BERTneglectsdependencybetweenthemaskedpositions\nandsuffersfromadiscrepancy.Inlightoftheseprosandcons,we\nproposeXLNet,ageneralizedautoregressivepretrainingmethodthat(1)enables\nlearningbidirectionalcontextsbymaximizingtheexpectedlikelihoodoverall\npermutationsofthefactorizationorderand(2)overcomesthelimitationsofBERT\nthankstoitsautoregressiveformulation.Furthermore,XLNetintegratesideas\nfromTransformer-XL,thestate-of-the-artautoregressivemodel,intop

In [5]:
doc = fitz.open(path_1)
print ("number of pages: %i" % doc.pageCount)
print(doc.metadata)

number of pages: 13
{'format': 'PDF 1.4', 'title': 'Climate change now detectable from any single day of weather at global scale', 'author': 'Sebastian Sippel', 'subject': 'Nature Climate Change, doi:10.1038/s41558-019-0666-7', 'keywords': None, 'creator': 'Springer', 'producer': None, 'creationDate': "D:20191220113831+05'30'", 'modDate': "D:20191220113925+05'30'", 'encryption': None}


In [92]:
def get_pages(path):
    '''
    This function will look at each page of the paper and if it find the occurence of 
    the word data in the corresponding page, it will store it in the json file so we can
    later on, hand label it as relevant or unrelevant to our task
    path: the path of the paper we are interested in
    '''
    doc = fitz.open(path)
    for i in range(doc.pageCount):
        page = doc.getPage(i)
        page = re.sub(r'\n', '', page.extractText())
        if 'data' in page:
            if not os.path.exists('pages_to_validate'): os.makedirs('pages_to_validate', exist_ok=True)
            nme = 'pages_to_validate/{}_page_{}.json'.format(path, i)
            with open(nme, 'w') as raw: 
                json.dump(page, raw, indent=4, sort_keys=False)
    return

In [7]:
#we create a function to extract mention of dataset
#the goal is then to review the pages correspoding and to label them a relevant of not for the SDGs
def get_relevant_pages(paths):
    '''
    This function extract the page from papers that potentially have the mention of a dataset
    paths: all the path for the papers that we want to consider
    '''
    

In [93]:
def process_paper(self, dict_):
    #1st: Extract Metadata
    try: self.title = dict_['title']
    except: self.title = np.nan
    try: self.authors = dict_['authors'].values()
    except: self.authors = np.nan
    try: self.year = dict_['year']
    except: self.title = np.nan
    try: self.citations = dict_['n_citation']
    except: self.citations = np.nan
        
    #2nd: Analyse the paper and extract pages with mention of 'data'
    self.url = dict_['url'][0]
    os.chdir('/Users/spezzata/Documents/Projects/AI4Good/data_aiminer/papers') 
    urlretrieve(url, "{}".format(self.url))
    path = "{}".format(self.url)
    get_pages(path)
    os.remove(path)
    
    #3rd: Store the information in a json file 
    #os.chdir(dir_for_papers) ##TODO define dir_for_info
    #os.makedirs('{}', exist_ok=True)
    
    return

In [94]:
def test_url(dict_):
    url_ = dict_['url'][0]
    try:
        if url_[-3:] == 'pdf':
            process_paper(dict_)
        else: pass
    except: pass
    return

In [95]:
#TODO: make a function that can process in parallel all of the papers from the aiminer doc that you have

In [15]:
for i in range(5):
    fileline = file.readline()
    if (fileline == ''): #end of file reached
        break
    print(fileline)

{"id": "53a7280320f7420be8ba5e96", "title": "Parsing.", "authors": [{"name": "Ralph Grishman"}], "venue": {"raw": "ACL", "id": "547ffa8cdabfaebedf84f2ad"}, "year": 1981, "n_citation": 11, "page_start": "", "page_end": "", "lang": "en", "volume": "", "issue": "", "pdf": "//static.aminer.org/pdf/20160902/aclanthology/aclweb/ACL-1981-6538.pdf", "url": ["http://aclweb.org/anthology-new/P/P81/P81-1022.pdf", "https://static.aminer.org/pdf/20160902/aclanthology/index.txt", "https://static.aminer.org/pdf/20170130/aclanthology/index.txt"]}

{"id": "53a7293220f7420be8bcb112", "title": "Quiet calls: talking silently on mobile phones.", "authors": [{"name": "Sara A. Bly"}, {"name": "Tomas Sokoler", "id": "562cd13d45cedb3398cd2b14"}, {"name": "Les Nelson", "id": "53f4442fdabfaedf435c3ea2"}], "venue": {"raw": "CHI"}, "year": 2001, "n_citation": 0, "page_start": "174", "page_end": "187", "lang": "en", "volume": "", "issue": "", "url": ["http://www.acm.org/pubs/citations/proceedings/chi/365024/p174-bl

In [12]:
for line in open("aminer_papers_0/aminer_papers_0.txt")[:20]:
    print(line)

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [32]:
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir()

File Name                                             Modified             Size
aminer_papers_0.txt                            2019-01-20 20:56:00  10000003049
aminer_papers_1.txt                            2019-01-20 21:10:42  10000001184
aminer_papers_2.txt                            2019-01-20 21:26:16  10000004163
aminer_papers_3.txt                            2019-01-20 21:41:50  10000001016


In [33]:
file1 = open("aminer_papers_0/aminer_papers_0.txt","r")

In [29]:
file1 = open("MyFile.txt","r")

In [ ]:
file1.read()

In [7]:
start = time.perf_counter()
page1 = doc.loadPage(0)
page1text = page1.getText("page")
end = time.perf_counter()
print(page1text)

Letters
https://doi.org/10.1038/s41558-019-0666-7
1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland. 3Norwegian Institute 
of Bioeconomy Research, Ås, Norway. 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: sebastian.sippel@env.ethz.ch
For generations, climate scientists have educated the public 
that ‘weather is not climate’, and climate change has been 
framed as the change in the distribution of weather that slowly 
emerges from large variability over decades1–7. However, 
weather when considered globally is now in uncharted terri-
tory. Here we show that on the basis of a single day of globally 
observed temperature and moisture, we detect the fingerprint 
of externally driven climate change, and conclude that Earth as 
a whole is warming. Our detection approach invokes statisti-
cal learning and climate model simulations to encapsulate the 
relationship between 

In [8]:
page1text

'Letters\nhttps://doi.org/10.1038/s41558-019-0666-7\n1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland. 3Norwegian Institute \nof Bioeconomy Research, Ås, Norway. 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: sebastian.sippel@env.ethz.ch\nFor generations, climate scientists have educated the public \nthat ‘weather is not climate’, and climate change has been \nframed as the change in the distribution of weather that slowly \nemerges from large variability over decades1–7. However, \nweather when considered globally is now in uncharted terri-\ntory. Here we show that on the basis of a single day of globally \nobserved temperature and moisture, we detect the fingerprint \nof externally driven climate change, and conclude that Earth as \na whole is warming. Our detection approach invokes statisti-\ncal learning and climate model simulations to encapsulate the \nrelati

In [40]:
print(end-start)

0.022048476000236406


In [38]:
for block in page1.getText("blocks"):
        print(block)

(469.82708740234375, 26.17009925842285, 544.9070434570312, 52.01009750366211, 'Letters', 0, 0)
(391.9768981933594, 52.054752349853516, 543.9119262695312, 62.38275146484375, 'https://doi.org/10.1038/s41558-019-0666-7', 1, 0)
(42.51969909667969, 726.0870361328125, 552.6649780273438, 747.8230590820312, '1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland. 3Norwegian Institute \nof Bioeconomy Research, Ås, Norway. 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: sebastian.sippel@env.ethz.ch', 2, 0)
(42.49089813232422, 199.68580627441406, 293.76123046875, 706.2036743164062, 'For generations, climate scientists have educated the public \nthat ‘weather is not climate’, and climate change has been \nframed as the change in the distribution of weather that slowly \nemerges from large variability over decades1–7. However, \nweather when considered globally is now in uncharted ter

In [35]:
page1text

'Letters\nhttps://doi.org/10.1038/s41558-019-0666-7\n1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland. 3Norwegian Institute \nof Bioeconomy Research, Ås, Norway. 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: sebastian.sippel@env.ethz.ch\nFor generations, climate scientists have educated the public \nthat ‘weather is not climate’, and climate change has been \nframed as the change in the distribution of weather that slowly \nemerges from large variability over decades1–7. However, \nweather when considered globally is now in uncharted terri-\ntory. Here we show that on the basis of a single day of globally \nobserved temperature and moisture, we detect the fingerprint \nof externally driven climate change, and conclude that Earth as \na whole is warming. Our detection approach invokes statisti-\ncal learning and climate model simulations to encapsulate the \nrelati

In [8]:
page1.getText("blocks")

[(469.82708740234375,
  26.17009925842285,
  544.9070434570312,
  52.01009750366211,
  'Letters',
  0,
  0),
 (391.9768981933594,
  52.054752349853516,
  543.9119262695312,
  62.38275146484375,
  'https://doi.org/10.1038/s41558-019-0666-7',
  1,
  0),
 (42.51969909667969,
  726.0870361328125,
  552.6649780273438,
  747.8230590820312,
  '1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland. 3Norwegian Institute \nof Bioeconomy Research, Ås, Norway. 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: sebastian.sippel@env.ethz.ch',
  2,
  0),
 (42.49089813232422,
  199.68580627441406,
  293.76123046875,
  706.2036743164062,
  'For generations, climate scientists have educated the public \nthat ‘weather is not climate’, and climate change has been \nframed as the change in the distribution of weather that slowly \nemerges from large variability over decades1–7. However, \nweath

In [4]:
doc = open(path_1, mode='rb')
pdf_document = PyPDF4.PdfFileReader(doc)

In [43]:
first_page = pdf_document.getPage(0)
first_page.extractText()

'1\nInstitute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. \n2\nSeminar for Statistics, ETH Zurich, Zurich, Switzerland. \n3\nNorwegian Institute \nof Bioeconomy Research, \n˜\ns, Norway. \n4\nSwiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: \nsebastian.sippel@env.ethz.ch\nFor generations, climate scientists have educated the public \nthat ‚weather is not climate™, and climate change has been \nframed as the change in the distribution of weather that slowly \nemerges from large variability over decades\n1\nŒ\n7\n. However, \nweather when considered globally is now in uncharted terri\n-\ntory. Here we show that on the basis of a single day of globally \nobserved temperature and moisture, we detect the fingerprint \n\nof externally driven climate change, and conclude that Earth as \na whole is warming. Our detection approach invokes statisti\n-\ncal learning and climate model simulations to encapsulate the \n\nrelationship between s

In [47]:
first_page_ = re.sub(r'\n', '', first_page.extractText())

In [49]:
first_page_

'1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland. 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland. 3Norwegian Institute of Bioeconomy Research, ˜s, Norway. 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland. *e-mail: sebastian.sippel@env.ethz.chFor generations, climate scientists have educated the public that ‚weather is not climate™, and climate change has been framed as the change in the distribution of weather that slowly emerges from large variability over decades1Œ7. However, weather when considered globally is now in uncharted terri-tory. Here we show that on the basis of a single day of globally observed temperature and moisture, we detect the fingerprint of externally driven climate change, and conclude that Earth as a whole is warming. Our detection approach invokes statisti-cal learning and climate model simulations to encapsulate the relationship between spatial patterns of daily temperature and humidity, and key c

In [50]:
first_page_.find('difference')

3336

In [52]:
first_page_sentences = first_page_.split('.')

In [51]:
first_page_[3336:]

'd'

In [58]:
first_page_sentences[2].find('Norway')

50

In [53]:
first_page_sentences

['1Institute for Atmospheric and Climate Science, ETH Zurich, Zurich, Switzerland',
 ' 2Seminar for Statistics, ETH Zurich, Zurich, Switzerland',
 ' 3Norwegian Institute of Bioeconomy Research, ˜s, Norway',
 ' 4Swiss Data Science Center, ETH Zurich and EPFL, Lausanne, Switzerland',
 ' *e-mail: sebastian',
 'sippel@env',
 'ethz',
 'chFor generations, climate scientists have educated the public that ‚weather is not climate™, and climate change has been framed as the change in the distribution of weather that slowly emerges from large variability over decades1Œ7',
 ' However, weather when considered globally is now in uncharted terri-tory',
 ' Here we show that on the basis of a single day of globally observed temperature and moisture, we detect the fingerprint of externally driven climate change, and conclude that Earth as a whole is warming',
 ' Our detection approach invokes statisti-cal learning and climate model simulations to encapsulate the relationship between spatial patterns of 

In [54]:
mentions = set()

In [55]:
def get_extract_metion(sentence):
    if sentence.find('data')!=-1:
        metions.add(''.join())
    return

In [ ]:
map(get_extract_metion, first_page_sentences)